In [4]:
app.run(jupyter_mode="inline")


<IPython.core.display.Javascript object>

In [3]:
!pip install dash jupyter-dash
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Load and prepare main data
df = pd.read_csv("/State_Healthcare_Facilities_Data.csv")
df['Lower CL'] = pd.to_numeric(df['Lower CL'], errors='coerce')
df['Upper CL'] = pd.to_numeric(df['Upper CL'], errors='coerce')
df['Observed'] = pd.to_numeric(df['Observed'], errors='coerce')
df['Predicted'] = pd.to_numeric(df['Predicted'], errors='coerce')
df['Average CL'] = df[['Lower CL', 'Upper CL']].mean(axis=1)

# Calculate Observed/Predicted and Significant flag
df["Observed / Predicted"] = df["Observed"] / df["Predicted"]
df["Significant"] = ~df.apply(lambda row: row["Lower CL"] <= 1 <= row["Upper CL"], axis=1)

# Load additional data
saar_2023 = pd.read_csv("/2023 SAAR Data - Sheet1.csv")
saar_forecast_2024 = pd.read_csv("/forecasted_saar_2024.csv")
sir_2022 = pd.read_csv("/2022 SIR Data (Acute Care Hospitals ONLY) - Sheet1.csv")
forecast_pctNS_2021 = pd.read_csv("/forecasted_pctNS_2021.csv")
sir_forecast_2022 = pd.read_csv("/sir_forecast_2022.csv")

# Rename for clarity
saar_2023 = saar_2023.rename(columns={'state': 'State Abbreviation', 'saar': 'SAAR 2023'})
saar_forecast_2024 = saar_forecast_2024.rename(columns={'state': 'State Abbreviation', 'Forecasted_saar_2024': 'SAAR Forecast 2024'})
sir_2022 = sir_2022.rename(columns={'state': 'State Abbreviation', 'pctNS': 'SIR 2022'})
forecast_pctNS_2021 = forecast_pctNS_2021.rename(columns={'state': 'State Abbreviation', 'Forecasted_pctNS_2021': 'Forecasted % NS 2021'})
sir_forecast_2022 = sir_forecast_2022.rename(columns={'state': 'State Abbreviation', 'sir': 'Forecasted SIR 2022'})

# Full list of U.S. states
all_states = pd.DataFrame({
    'State Abbreviation': [
        'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
        'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
        'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
    ]
})

# Merge all data into one frame
df_cl = all_states.merge(
    df[['State Abbreviation', 'Lower CL', 'Upper CL', 'Average CL', 'Observed / Predicted', 'Significant']],
    on='State Abbreviation', how='left'
).merge(
    saar_2023[['State Abbreviation', 'SAAR 2023']],
    on='State Abbreviation', how='left'
).merge(
    saar_forecast_2024[['State Abbreviation', 'SAAR Forecast 2024']],
    on='State Abbreviation', how='left'
).merge(
    sir_2022[['State Abbreviation', 'SIR 2022']],
    on='State Abbreviation', how='left'
).merge(
    forecast_pctNS_2021[['State Abbreviation', 'Forecasted % NS 2021']],
    on='State Abbreviation', how='left'
).merge(
    sir_forecast_2022[['State Abbreviation', 'Forecasted SIR 2022']],
    on='State Abbreviation', how='left'
)

# Fill missing numeric values
df_cl[['Lower CL', 'Upper CL', 'Average CL', 'Observed / Predicted', 'SAAR 2023', 'SAAR Forecast 2024', 'SIR 2022', 'Forecasted % NS 2021', 'Forecasted SIR 2022']] = df_cl[[
    'Lower CL', 'Upper CL', 'Average CL', 'Observed / Predicted', 'SAAR 2023', 'SAAR Forecast 2024', 'SIR 2022', 'Forecasted % NS 2021', 'Forecasted SIR 2022'
]].fillna(-1)

# Create Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("U.S. State-Level Antibiotic Resistance Map", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='cl-selector',
        options=[
            {'label': 'Observed / Predicted Infections ', 'value': 'Observed / Predicted'},
            {'label': 'Actual/ Predicted Infections Average Confidence Level', 'value': 'Average CL'},
            {'label': 'Upper CL of Actual/ Predicted Infections', 'value': 'Upper CL'},
            {'label': 'Lower CL of Actual/ Predicted Infections', 'value': 'Lower CL'},
            {'label': '2023 SAAR', 'value': 'SAAR 2023'},
            {'label': 'Forecasted SAAR 2024', 'value': 'SAAR Forecast 2024'},
            {'label': 'SIR 2022', 'value': 'SIR 2022'},
            {'label': 'Forecasted % NS 2021', 'value': 'Forecasted % NS 2021'},
            {'label': 'Forecasted SIR 2023', 'value': 'Forecasted SIR 2022'},
        ],
        value='Average CL',
        clearable=False,
        style={
            'width': '40%',
            'margin': 'auto',
            'fontSize': '14px'
        }
    ),
    dcc.Graph(id='us-map')
])

# Callback
@app.callback(
    Output('us-map', 'figure'),
    Input('cl-selector', 'value')
)
def update_map(selected_cl):
    threshold = 1
    min_val = df_cl[df_cl[selected_cl] != -1][selected_cl].min()
    max_val = df_cl[df_cl[selected_cl] != -1][selected_cl].max()
    threshold_norm = (threshold - min_val) / (max_val - min_val) if max_val != min_val else 0.5

    valid_data = df_cl[df_cl[selected_cl] != -1]
    missing_data = df_cl[df_cl[selected_cl] == -1]

    fig = px.choropleth(
        valid_data,
        locations='State Abbreviation',
        locationmode='USA-states',
        color=selected_cl,
        scope='usa',
        hover_name='State Abbreviation',
        labels={selected_cl: f"{selected_cl} Ratio"},
        title=f"{selected_cl} Per State"
    )

    fig.update_traces(
        colorscale=[
            [0.0, "#f2e6f9"],
            [threshold_norm * 0.99, "#4b0082"],
            [threshold_norm, "#FFA500"],
            [1.0, "#FFA500"]
        ],
        marker_line_color='white',
        marker_line_width=1,
        colorbar=dict(
            thickness=10,
            len=0.4,
            tickvals=[min_val, threshold, max_val],
            ticktext=[
                f"{min_val:.2f}",
                "Threshold (1.00)",
                f"{max_val:.2f}"
            ],
            title=f"{selected_cl} value",
            tickfont=dict(size=10),
            titlefont=dict(size=12),
            borderwidth=1,
            bordercolor='gray'
        )
    )

    fig.add_choropleth(
        locations=missing_data['State Abbreviation'],
        locationmode='USA-states',
        z=[-1] * len(missing_data),
        showscale=False,
        name="Insufficient Data",
        hovertemplate="<b>%{location}</b><br>Insufficient data<extra></extra>",
        marker_line_color='white',
        marker_line_width=1,
        colorscale=[[0, "#d3d3d3"], [1, "#d3d3d3"]],
        zmin=-1,
        zmax=0
    )

    fig.update_coloraxes(
        colorscale=[
            [0.0, "#fff3b7"],
            [threshold_norm * 0.99, "#f48c06"],
            [threshold_norm, "#d00000"],
            [1.0, "#d00000"]
        ],
        cmin=min_val,
        cmax=max_val
    )

    fig.update_layout(
        height=600,
        width=900,
        margin=dict(l=10, r=10, t=40, b=10),
        font=dict(family="Tahoma", size=12, color="black"),
        title_font=dict(family="Tahoma", size=20, color="black"),
        legend=dict(
            title="",
            font=dict(size=12),
            itemsizing='constant',
            orientation='v',
            x=0.82,
            y=0.1,
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='black',
            borderwidth=1
        )
    )

    return fig

# Run the app (in Jupyter)
app.run(jupyter_mode="inline")


<IPython.core.display.Javascript object>